In [1]:
# Check that we have a GPU
!nvidia-smi

Fri Jul 10 00:51:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   32C    P8    10W / 275W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0 Off |                  N/A |
|  0%   31C    P8    10W / 275W |     18MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [3]:
!mkdir ../models
!mkdir ../models/smallBERTa

mkdir: cannot create directory ‘../models’: File exists
mkdir: cannot create directory ‘../models/smallBERTa’: File exists


### config arguments
<pre>Args:
vocab_size (:obj:`int`, optional, defaults to 30522):
    Vocabulary size of the BERT model. Defines the different tokens that
    can be represented by the `inputs_ids` passed to the forward method of :class:`~transformers.BertModel`.
hidden_size (:obj:`int`, optional, defaults to 768):
    Dimensionality of the encoder layers and the pooler layer.
num_hidden_layers (:obj:`int`, optional, defaults to 12):
    Number of hidden layers in the Transformer encoder.
num_attention_heads (:obj:`int`, optional, defaults to 12):
    Number of attention heads for each attention layer in the Transformer encoder.
intermediate_size (:obj:`int`, optional, defaults to 3072):
    Dimensionality of the "intermediate" (i.e., feed-forward) layer in the Transformer encoder.
hidden_act (:obj:`str` or :obj:`function`, optional, defaults to "gelu"):
    The non-linear activation function (function or string) in the encoder and pooler.
    If string, "gelu", "relu", "swish" and "gelu_new" are supported.
hidden_dropout_prob (:obj:`float`, optional, defaults to 0.1):
    The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler.
attention_probs_dropout_prob (:obj:`float`, optional, defaults to 0.1):
    The dropout ratio for the attention probabilities.
max_position_embeddings (:obj:`int`, optional, defaults to 512):
    The maximum sequence length that this model might ever be used with.
    Typically set this to something large just in case (e.g., 512 or 1024 or 2048).
type_vocab_size (:obj:`int`, optional, defaults to 2):
    The vocabulary size of the `token_type_ids` passed into :class:`~transformers.BertModel`.
initializer_range (:obj:`float`, optional, defaults to 0.02):
    The standard deviation of the truncated_normal_initializer for initializing all weight matrices.
layer_norm_eps (:obj:`float`, optional, defaults to 1e-12):
    The epsilon used by the layer normalization layers.
gradient_checkpointing (:obj:`bool`, optional, defaults to False):
    If True, use gradient checkpointing to save memory at the expense of slower backward pass.
</pre>

In [4]:
import json
config = {
    "model_type":"bert",
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.3,
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 6,
    "vocab_size": 5000,
    "intermediate_size": 256,
    "max_position_embeddings": 256
}

with open("../models/smallBERTa/config.json", 'w') as fp:
    json.dump(config, fp)

In [5]:
train_path='../wikitext-2-raw/wiki.train.raw'
TEST_FILE='../wikitext-2-raw/wiki.test.raw'
eval_path= '../wikitext-2-raw/wiki.valid.raw'

In [6]:

import os
#Setting environment variables
os.environ["train_path"] = train_path
os.environ["eval_path"] = eval_path
os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)
weights_dir = "../models/smallBERTa/weights"
!mkdir {weights_dir}

mkdir: cannot create directory ‘../models/smallBERTa/weights’: File exists


In [7]:
# from tokenizers import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer()
# tokenizer.train([train_path], vocab_size=20000)
# !export TOKENIZERS_PARALLELISM=false
# from tokenizers import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing
# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# vocab_size=5000
# # Customize training
# tokenizer.train(files=train_path, vocab_size=vocab_size, min_frequency=5, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])

In [8]:
# tokenizer.save("../models/smallBERTa", "smallBERTa")

TypeError: 

In [9]:
# !mv ../models/smallBERTa/smallBERTa-vocab.json ../models/smallBERTa/vocab.json
# !mv ../models/smallBERTa/smallBERTa-merges.txt ../models/smallBERTa/merges.txt

mv: cannot stat '../models/smallBERTa/smallBERTa-vocab.json': No such file or directory
mv: cannot stat '../models/smallBERTa/smallBERTa-merges.txt': No such file or directory


In [10]:
cmd = '''python ../transformers/examples/language-modeling/run_language_modeling.py --output_dir {0}  \
    --model_type bert \
    --mlm \
    --train_data_file {1} \
    --eval_data_file {2} \
    --config_name ../models/smallBERTa \
    --tokenizer_name bert-base-uncased \
    --do_train \
    --line_by_line \
    --overwrite_output_dir \
    --do_eval \
    --block_size 256 \
    --learning_rate 1e-4 \
    --num_train_epochs 1 \
    --save_total_limit 2 \
    --save_steps 10000 \
    --logging_steps 1000 \
    --per_gpu_eval_batch_size 32 \
    --per_gpu_train_batch_size 32 \
    --evaluate_during_training \
    --seed 42 \
    '''.format(weights_dir, train_path, eval_path)

In [ ]:
#from transformers import DataCollatorForPermutationLanguageModeling
!{cmd}

07/10/2020 00:51:50 - INFO - transformers.training_args -   PyTorch: setting up devices
07/10/2020 00:51:50 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 2, distributed training: False, 16-bits training: False
07/10/2020 00:51:50 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../models/smallBERTa/weights', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=32, per_gpu_eval_batch_size=32, gradient_accumulation_steps=1, learning_rate=0.0001, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jul10_00-51-50_fritz', logging_first_step=False, logging_steps=1000, save_steps=10000, save_total_limit=2, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=Fal